Importing Libraries

In [0]:
# -*- coding: utf-8 -*-
"""
Created on Wed Apr 22 09:57:27 2020

@author: Subham
"""


import pandas as pd
import numpy as np
import os
import sys

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Dense, Input, Flatten, Conv1D, MaxPooling1D, Embedding, Conv2D, MaxPool2D
from keras.models import Model
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import gensim

import keras.regularizers

Using TensorFlow backend.


In [0]:
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

Importing Training and Testing Dataset

In [0]:
data_train=pd.read_csv('drive/My Drive/dataset_train.csv',encoding = "ISO-8859-1")
data_train_target1=data_train[data_train["Target"]=="Atheism"]
data_train_target2=data_train[data_train["Target"]=="Climate Change is a Real Concern"]
data_train_target3=data_train[data_train["Target"]=="Feminist Movement"]
data_train_target4=data_train[data_train["Target"]=="Hillary Clinton"]
data_train_target5=data_train[data_train["Target"]=="Legalization of Abortion"]

data_test=pd.read_csv('drive/My Drive/dataset_test.csv',encoding = "ISO-8859-1")
data_test_target1=data_test[data_test["Target"]=="Atheism"]
data_test_target2=data_test[data_test["Target"]=="Climate Change is a Real Concern"]
data_test_target3=data_test[data_test["Target"]=="Feminist Movement"]
data_test_target4=data_test[data_test["Target"]=="Hillary Clinton"]
data_test_target5=data_test[data_test["Target"]=="Legalization of Abortion"]


target1_test_len=len(data_test_target1)
target2_test_len=len(data_test_target2)
target3_test_len=len(data_test_target3)
target4_test_len=len(data_test_target4)
target5_test_len=len(data_test_target5)

target1_train_len=len(data_train_target1)
target2_train_len=len(data_train_target2)
target3_train_len=len(data_train_target3)
target4_train_len=len(data_train_target4)
target5_train_len=len(data_train_target5)

print(target4_train_len)
print(target4_test_len)

639
295


Defining Functions for Pre-processing of Data

In [0]:
def remove_punctuations(docs): #list of strings
    punctuations = '''!()[]{};:"\,<>'./?@#$%^&*_~-'''
    docs2=[];
    i=0;
    for sent in docs:
        s=""
        for x in sent:
            if(x not in punctuations):
                s=s+x;
        docs2.append(s);


    docs=docs2;
    docs2=[]
    return docs


def convert_lowercase(docs): #list of strings
    docs2=[]
    for sent in docs:
        sent=sent.lower()
        docs2.append(sent);

    docs=docs2;
    docs2=[]
    return docs




def tokenize_data(docs): #list of strings
    docs2=[]
    for sent in docs:
        sent=word_tokenize(sent)
        docs2.append(sent);

    docs=docs2;
    docs2=[]
    return docs

def remove_stopwords(docs): #list of list of words as param
    stop_words=set(stopwords.words("english"));

    docs2=[]
    for sent in docs:
        s=[];
        for word in sent:
            if(word not in stop_words):
                s.append(word);
        docs2.append(s);

    docs=docs2;
    docs2=[]
    return docs


def make_string(docs): #list of list of words as param
    docs2=[]
    for sent in docs:
        s=""
        s=' '.join(sent)
        docs2.append(s);

    docs=docs2;
    docs2=[]
    return docs


def lemmatize(docs): #list of list of words as param
    lemmatizer=WordNetLemmatizer();
    docs2=[]
    for sent in docs:
        s=[];
        for word in sent:
            s.append(lemmatizer.lemmatize(word));
        docs2.append(s);

    docs=docs2;
    docs2=[]
    return docs


def remove_digits(docs): #list of strings
    docs2=[]
    for sent in docs:
        sent2=sent.split(' ');
        words=[]
        for x in sent2:
            flag=0;
            for c in x:
                if(c>='0' and c<='9'):
                    flag=1;
                    break;
                else:
                    pass;
            if(flag==0):
                words.append(x);
        words=' '.join(words);
        docs2.append(words)
    docs=docs2;
    docs2=[]
    return docs


Pre-processing of Data

In [0]:


target1_train_data=[]
for i in range(0,target1_train_len):
    target1_train_data.append(data_train_target1["Tweet"][i])
    

print(len(target1_train_data))

target1_train_data=convert_lowercase(target1_train_data)
target1_train_data=remove_digits(target1_train_data)
target1_train_data=remove_punctuations(target1_train_data)
target1_train_data=tokenize_data(target1_train_data)
target1_train_data=remove_stopwords(target1_train_data)
target1_train_data=lemmatize(target1_train_data)
target1_train_data=make_string(target1_train_data)


data_test=pd.read_csv('drive/My Drive/dataset_test.csv',encoding = "ISO-8859-1")
data_test.head()
data_test_target1=data_test[data_test["Target"]=="Atheism"]

target1_test_len=len(data_test_target1)

target1_test_data=[]
for i in range(0,target1_test_len):
    target1_test_data.append(data_test_target1["Tweet"][i])
    
target1_test_data=convert_lowercase(target1_test_data)
target1_test_data=remove_digits(target1_test_data)
target1_test_data=remove_punctuations(target1_test_data)
target1_test_data=tokenize_data(target1_test_data)
target1_test_data=remove_stopwords(target1_test_data)
target1_test_data=lemmatize(target1_test_data)
target1_test_data=make_string(target1_test_data)


y1=[]
for i in range(0,target1_train_len):
    y1.append(data_train_target1["Stance"][i])
    

target1_full_data=[]
target1_full_data.extend(target1_train_data)
target1_full_data.extend(target1_test_data)


513


Loading Pre-Trained Word2Vec Model of dimension 300

In [0]:
word2vec = gensim.models.KeyedVectors.load_word2vec_format('./drive/My Drive/GoogleNews-vectors-negative300.bin', binary=True)  

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [0]:
len(word2vec.wv["stop"])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


300

Creating Dictionary for mapping Words to indices

In [0]:
# tokenize the words in the texts
max_nb_words=2000
tokenizer = Tokenizer(num_words = max_nb_words) 
tokenizer.fit_on_texts(target1_full_data) 
# convert each review text into a sequence of word-indices
matrix_word_indices = tokenizer.texts_to_sequences(target1_full_data)
# the dictionary for mapping a word to an index
dictionary_word_index = tokenizer.word_index

In [0]:
print(len(dictionary_word_index))

2892


In [0]:
print(len(matrix_word_indices))

733


Padding sentences to a fixed length

In [0]:
# pad each review text to a fixed length of word sequence
num_words_per_review=20
matrix_word_indices_fixed_length = pad_sequences(matrix_word_indices, maxlen = num_words_per_review)
# convert to numpy arrays 
data = np.array(matrix_word_indices_fixed_length)

In [0]:
print(data[0])

[  0   0 282   7 150   4 283 363 364  98   7  39 228 922 365  15 923 112
  40   1]


In [0]:
print(matrix_word_indices_fixed_length.shape)

(733, 20)


In [0]:
# allocation of training data and validation data
x_train = data[:len(data_train_target1)]
x_test = data[len(data_train_target1):]

Creating One-hot encoding of Target, Against- 100, Favor- 010 , None- 001

In [0]:
y_train=np.zeros(shape=(len(data_train_target1),3))

for i in range(len(y_train)):
  if(y1[i]=='AGAINST'):
    y_train[i][0]=1;
  elif(y1[i]=='FAVOR'):
    y_train[i][1]=1;
  else:
    y_train[i][2]=1;

Seperation of Training and Testing Set

In [0]:
print(len(x_train))
print(len(x_test))

513
220


In [0]:
print(x_train.shape)

(513, 20)


Creating Word Embedding Matrix of dimension N X 300, N is the no. of words

In [0]:
# prepare embedding matrix
num_words = len(dictionary_word_index)
# embedding_matrix[0] is a all-zero vector representing no word
embedding_matrix = np.zeros((num_words+1, 300)) 

In [0]:
init_vector=np.zeros(300)

In [0]:
unknown_words=[]
for word, index in dictionary_word_index.items():
    if index > max_nb_words:
        continue 
    # get the glove vector for the word
    try:
      word_vector = model.wv[word]
      if word_vector is not None: 
          embedding_matrix[index] = word_vector
    except:
      unknown_words.append(word)
      embedding_matrix[index] = init_vector
  
print("The no. of unknown words: ", len(unknown_words))

The no. of unknown words:  352


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  import sys


In [0]:
print(len(embedding_matrix))

2893


Defining and Training model

In [0]:

# define the model
# layer 0: the input layer
num_words_per_tweet=20
sequence_input = Input(shape=(num_words_per_tweet,), dtype='int32')
# layer-1: the embedding layer
embedding_layer = Embedding(num_words+1, 300, weights=[embedding_matrix], input_length=num_words_per_tweet, trainable=True)
embedded_output = embedding_layer(sequence_input)
# layer-2: the first convolution layer
x = Conv1D(nb_filter=128, filter_length=2, activation='relu')(embedded_output)
# layer-3: the first pooling layer
x = MaxPooling1D(pool_length=2)(x)
# layer-4: the second convolution layer
x = Conv1D(128, 2, activation='relu')(x)
# layer-5: the second pooling layer
x = MaxPooling1D(pool_length = 2)(x)
# flatten layer
x = Flatten()(x)
# layer-6: the first dense layer
x = Dense(output_dim = 128, activation='relu')(x)
# layer-7: the second dense layer
x = Dense(output_dim = 128, activation='relu')(x)
# layer-8: the output layer
final_output = Dense(3, activation='sigmoid')(x)

# define the model
model = Model(input=sequence_input, output=final_output)

#view model
print(model.summary())

# compile the model
model.compile(loss='categorical_crossentropy', optimizer='adagrad', metrics=['acc'])
# training and validation
print('Training the model ...')
model.fit(x=x_train, y=y_train, nb_epoch=15, batch_size=128, verbose=1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", filters=128, kernel_size=2)`
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=2)`
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=2)`
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=128)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=128)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: UserWarning: Update your `Model` call to the Keras 2 API: `

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 20)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 20, 300)           867900    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 19, 128)           76928     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 9, 128)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 8, 128)            32896     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 4, 128)            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 512)               0   

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/15
513/513 [==============================] - 1s 1ms/step - loss: 1.0341 - acc: 0.5010
Epoch 2/15
513/513 [==============================] - 0s 445us/step - loss: 1.2105 - acc: 0.5926
Epoch 3/15
513/513 [==============================] - 0s 459us/step - loss: 0.8099 - acc: 0.6745
Epoch 4/15
513/513 [==============================] - 0s 472us/step - loss: 0.6174 - acc: 0.7563
Epoch 5/15
513/513 [==============================] - 0s 471us/step - loss: 0.4738 - acc: 0.7778
Epoch 6/15
513/513 [==============================] - 0s 500us/step - loss: 0.7012 - acc: 0.7251
Epoch 7/15
513/513 [==============================] - 0s 450us/step - loss: 0.3257 - acc: 0.8187
Epoch 8/15
513/513 [==============================] - 0s 442us/step - loss: 0.2479 - acc: 0.8246
Epoch 9/15
513/513 [==============================] - 0s 457us/step - loss: 0.1796 - acc: 0.8772
Epoch 10/15
513/513 [==============================] - 0s 480us/step - loss: 0.0796 - acc: 0.9883
Epoch 11/15
513/513 [==========

Storing the Ground Truth value for final evaluation

In [0]:
data_test_answers=pd.read_csv('drive/My Drive/dataset_test_answers.csv')
data_test_target1_answers=data_test_answers[data_test_answers["Target"]=="Atheism"]
y_ground_truth=data_test_target1_answers["Stance"].values;


In [0]:
print(y_ground_truth)

['AGAINST' 'AGAINST' 'AGAINST' 'AGAINST' 'AGAINST' 'AGAINST' 'AGAINST'
 'AGAINST' 'AGAINST' 'AGAINST' 'AGAINST' 'AGAINST' 'AGAINST' 'AGAINST'
 'AGAINST' 'AGAINST' 'AGAINST' 'AGAINST' 'AGAINST' 'AGAINST' 'AGAINST'
 'AGAINST' 'AGAINST' 'AGAINST' 'AGAINST' 'AGAINST' 'AGAINST' 'AGAINST'
 'AGAINST' 'AGAINST' 'AGAINST' 'AGAINST' 'AGAINST' 'AGAINST' 'AGAINST'
 'AGAINST' 'AGAINST' 'AGAINST' 'AGAINST' 'AGAINST' 'AGAINST' 'AGAINST'
 'AGAINST' 'AGAINST' 'AGAINST' 'AGAINST' 'AGAINST' 'AGAINST' 'AGAINST'
 'AGAINST' 'AGAINST' 'AGAINST' 'AGAINST' 'AGAINST' 'AGAINST' 'AGAINST'
 'AGAINST' 'AGAINST' 'AGAINST' 'AGAINST' 'AGAINST' 'AGAINST' 'AGAINST'
 'AGAINST' 'AGAINST' 'AGAINST' 'AGAINST' 'AGAINST' 'AGAINST' 'AGAINST'
 'AGAINST' 'FAVOR' 'NONE' 'AGAINST' 'AGAINST' 'AGAINST' 'AGAINST'
 'AGAINST' 'AGAINST' 'FAVOR' 'AGAINST' 'AGAINST' 'AGAINST' 'FAVOR'
 'AGAINST' 'AGAINST' 'NONE' 'AGAINST' 'AGAINST' 'AGAINST' 'AGAINST' 'NONE'
 'NONE' 'AGAINST' 'NONE' 'NONE' 'NONE' 'NONE' 'AGAINST' 'AGAINST'
 'AGAINST' 'FAVO

Mapping targets to indices, Against - 0, Favor - 1, None -2

In [0]:
y_test=np.zeros(len(data_test_target1))

for i in range(len(y_ground_truth)):
  if(y_ground_truth[i]=='AGAINST'):
    y_test[i]=0;
  elif(y_ground_truth[i]=='FAVOR'):
    y_test[i]=1;
  else:
    y_test[i]=2;


Prediction of Test Data

In [0]:
y_pred=model.predict(x_test)

In [0]:
print(y_pred)

[[4.10786152e-01 1.60637796e-02 2.02181041e-02]
 [2.90657133e-01 4.93154330e-05 1.38823384e-06]
 [6.96861863e-01 2.33717983e-05 4.19936578e-06]
 [1.75553858e-01 1.49313509e-02 2.73184087e-05]
 [1.28371358e-01 2.45370567e-02 3.55809927e-04]
 [2.02653110e-02 2.80261040e-04 1.77979469e-03]
 [4.33236361e-03 3.25621188e-01 2.01311707e-03]
 [3.58386278e-01 2.19821930e-04 3.87542605e-05]
 [1.46240592e-02 1.15836859e-02 2.86225021e-01]
 [6.66997731e-02 1.11630950e-04 7.57686469e-10]
 [3.17097008e-01 3.49792838e-03 1.22957528e-02]
 [4.40665317e-05 5.77410901e-05 3.83785099e-01]
 [3.68267745e-01 3.17305326e-04 1.27106905e-04]
 [9.84281301e-04 5.99261820e-02 5.54996729e-03]
 [7.45049119e-03 5.71787357e-03 1.22929432e-05]
 [6.97539330e-01 1.11922622e-03 1.24543905e-04]
 [9.40428019e-01 1.00303043e-06 1.04864206e-09]
 [8.69690776e-02 1.03363395e-03 5.04293052e-10]
 [9.08427477e-01 1.83969736e-04 1.81388987e-05]
 [4.09312010e-01 8.08417899e-05 7.34684290e-05]
 [7.61620700e-01 1.57013535e-03 6.918609

In [0]:
y=np.zeros(len(y_pred))
for i in range(len(y_pred)):
  y[i]=np.argmax(y_pred[i])

In [0]:
print(y_test)
print(y)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 2. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 2. 0. 0. 0. 0. 2. 2. 0. 2. 2.
 2. 2. 0. 0. 0. 1. 2. 2. 2. 0. 1. 0. 0. 0. 0. 0. 0. 2. 0. 0. 2. 0. 1. 0.
 1. 1. 1. 0. 0. 1. 2. 0. 0. 2. 1. 0. 0. 2. 1. 0. 0. 1. 0. 1. 1. 0. 0. 1.
 1. 1. 0. 1. 0. 1. 0. 0. 0. 0. 2. 0. 2. 1. 2. 0. 2. 0. 1. 0. 0. 0. 0. 0.
 2. 1. 1. 1. 2. 0. 2. 2. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 1. 2. 1. 0. 1. 0. 1. 1. 2. 0. 0. 2. 0. 0. 0. 2. 0. 0.
 1. 0. 1. 0.]
[0. 0. 0. 0. 0. 0. 1. 0. 2. 0. 0. 2. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 2. 0.
 1. 0. 2. 0. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 2. 2. 0.
 1. 0. 1. 2. 1. 1. 0. 1. 0. 0. 0. 0. 2. 0. 2. 0. 0. 0. 2. 0. 0. 0. 0. 1.
 0. 2. 0. 0. 0. 0. 0. 1. 2. 0. 0. 2. 0. 0. 0. 1. 0. 0. 0. 2. 2. 0. 0. 0.
 2. 0. 0. 0. 1. 0. 2. 0. 2. 0. 1. 0. 

Finding F1_Score , Precision_score, Recall_score, Accuracy_score

In [0]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
print(f1_score(y,y_test,average='macro'))
print(precision_score(y,y_test,average='macro'))
print(recall_score(y,y_test,average='macro'))
print(accuracy_score(y,y_test))

0.5814095201377232
0.5931547619047618
0.5721500721500722
0.7181818181818181


Repeating Training and Testing for other 4 Targets

In [0]:
#2nd model



# define the model
# layer 0: the input layer
num_words_per_tweet=20
sequence_input = Input(shape=(num_words_per_tweet,), dtype='int32')
# layer-1: the embedding layer
embedding_layer = Embedding(num_words+1, 300, weights=[embedding_matrix], input_length=num_words_per_tweet, trainable=True)
embedded_output = embedding_layer(sequence_input)
# layer-2: the first convolution layer
x = Conv1D(nb_filter=128, filter_length=2, activation='relu')(embedded_output)
# layer-3: the first pooling layer
x = MaxPooling1D(pool_length=2)(x)
# layer-4: the second convolution layer
x = Conv1D(128, 2, activation='relu')(x)
# layer-5: the second pooling layer
x = MaxPooling1D(pool_length = 2)(x)
# flatten layer
x = Flatten()(x)
# layer-6: the first dense layer
x = Dense(output_dim = 128, activation='relu')(x)
# layer-7: the second dense layer
x = Dense(output_dim = 128, activation='relu')(x)
# layer-8: the output layer
final_output = Dense(3, activation='sigmoid')(x)

# define the model
model = Model(input=sequence_input, output=final_output)

#view model
print(model.summary())

# compile the model
model.compile(loss='categorical_crossentropy', optimizer='adagrad', metrics=['acc'])
# training and validation
print('Training the model ...')
model.fit(x=x_train, y=y_train, nb_epoch=15, batch_size=128, verbose=1)




y_pred=model.predict(x_test)

y=np.zeros(len(y_pred))
for i in range(len(y_pred)):
  y[i]=np.argmax(y_pred[i])


from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
print(f1_score(y,y_test,average='macro'))
print(precision_score(y,y_test,average='macro'))
print(recall_score(y,y_test,average='macro'))
print(accuracy_score(y,y_test))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", filters=128, kernel_size=2)`
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=2)`
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=2)`
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=128)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=128)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: UserWarning: Update your `Model` call to the Keras 2 API: `

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 20)                0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 20, 300)           867900    
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 19, 128)           76928     
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 9, 128)            0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 8, 128)            32896     
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 4, 128)            0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 512)               0   

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/15
513/513 [==============================] - 0s 704us/step - loss: 1.0824 - acc: 0.6043
Epoch 2/15
513/513 [==============================] - 0s 429us/step - loss: 0.9360 - acc: 0.5926
Epoch 3/15
513/513 [==============================] - 0s 447us/step - loss: 0.7729 - acc: 0.6179
Epoch 4/15
513/513 [==============================] - 0s 478us/step - loss: 0.5540 - acc: 0.8031
Epoch 5/15
513/513 [==============================] - 0s 484us/step - loss: 0.4173 - acc: 0.8012
Epoch 6/15
513/513 [==============================] - 0s 448us/step - loss: 0.2124 - acc: 0.8538
Epoch 7/15
513/513 [==============================] - 0s 431us/step - loss: 0.0825 - acc: 0.9766
Epoch 8/15
513/513 [==============================] - 0s 403us/step - loss: 0.6135 - acc: 0.7758
Epoch 9/15
513/513 [==============================] - 0s 471us/step - loss: 0.1169 - acc: 0.9864
Epoch 10/15
513/513 [==============================] - 0s 436us/step - loss: 0.0376 - acc: 0.9981
Epoch 11/15
513/513 [========

In [0]:
# for Target 2

data_train=pd.read_csv('drive/My Drive/dataset_train.csv',encoding = "ISO-8859-1")
data_train_target2=data_train[data_train["Target"]=="Climate Change is a Real Concern"]


target2_train_len=len(data_train_target2)


target2_train_data=[]
for i in range(target1_train_len,target1_train_len+target2_train_len):
    target2_train_data.append(data_train_target2["Tweet"][i])
    
    
print(len(target2_train_data))

target2_train_data=convert_lowercase(target2_train_data)
target2_train_data=remove_digits(target2_train_data)
target2_train_data=remove_punctuations(target2_train_data)
target2_train_data=tokenize_data(target2_train_data)
target2_train_data=remove_stopwords(target2_train_data)
target2_train_data=lemmatize(target2_train_data)
target2_train_data=make_string(target2_train_data)


data_test=pd.read_csv('drive/My Drive/dataset_test.csv',encoding = "ISO-8859-1")
data_test.head()
data_test_target2=data_test[data_test["Target"]=="Climate Change is a Real Concern"]

target2_test_len=len(data_test_target2)

target2_test_data=[]
for i in range(target1_test_len,target1_test_len+target2_test_len):
    target2_test_data.append(data_test_target2["Tweet"][i])
    
target2_test_data=convert_lowercase(target2_test_data)
target2_test_data=remove_digits(target2_test_data)
target2_test_data=remove_punctuations(target2_test_data)
target2_test_data=tokenize_data(target2_test_data)
target2_test_data=remove_stopwords(target2_test_data)
target2_test_data=lemmatize(target2_test_data)
target2_test_data=make_string(target2_test_data)


y1=[]
for i in range(target1_train_len,target1_train_len+target2_train_len):
    y1.append(data_train_target2["Stance"][i])
    

target2_full_data=[]
target2_full_data.extend(target2_train_data)
target2_full_data.extend(target2_test_data)


395


In [0]:
# tokenize the words in the texts
max_nb_words=2000
tokenizer = Tokenizer(num_words = max_nb_words) 
tokenizer.fit_on_texts(target2_full_data) 
# convert each review text into a sequence of word-indices
matrix_word_indices = tokenizer.texts_to_sequences(target2_full_data)
# the dictionary for mapping a word to an index
dictionary_word_index = tokenizer.word_index

print(len(dictionary_word_index))

2681


In [0]:
# pad each review text to a fixed length of word sequence
num_words_per_review=20
matrix_word_indices_fixed_length = pad_sequences(matrix_word_indices, maxlen = num_words_per_review)
# convert to numpy arrays 
data = np.array(matrix_word_indices_fixed_length)

print(matrix_word_indices_fixed_length.shape)

# allocation of training data and validation data
x_train = data[:len(data_train_target2)]
x_test = data[len(data_train_target2):]

y_train=np.zeros(shape=(len(data_train_target2),3))

for i in range(len(y_train)):
  if(y1[i]=='AGAINST'):
    y_train[i][0]=1;
  elif(y1[i]=='FAVOR'):
    y_train[i][1]=1;
  else:
    y_train[i][2]=1;


print(len(x_train))
print(len(x_test))


# prepare embedding matrix
num_words = len(dictionary_word_index)
# embedding_matrix[0] is a all-zero vector representing no word
embedding_matrix = np.zeros((num_words+1, 300)) 

init_vector=np.zeros(300)


unknown_words=[]
for word, index in dictionary_word_index.items():
    if index > max_nb_words:
        continue 
    # get the glove vector for the word
    try:
      word_vector = word2vec.wv[word]
      if word_vector is not None: 
          embedding_matrix[index] = word_vector
    except:
      unknown_words.append(word)
      #print(word)
      embedding_matrix[index] = init_vector
  
print("The no. of unknown words: ", len(unknown_words))

print(len(embedding_matrix))

(564, 20)
395
169
The no. of unknown words:  416
2682


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:41: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


In [0]:

# define the model
# layer 0: the input layer
num_words_per_tweet=20
sequence_input = Input(shape=(num_words_per_tweet,), dtype='int32')
# layer-1: the embedding layer
embedding_layer = Embedding(num_words+1, 300, weights=[embedding_matrix], input_length=num_words_per_tweet, trainable=True)
embedded_output = embedding_layer(sequence_input)
# layer-2: the first convolution layer
x = Conv1D(nb_filter=128, filter_length=2, activation='relu')(embedded_output)
# layer-3: the first pooling layer
x = MaxPooling1D(pool_length=2)(x)
# layer-4: the second convolution layer
x = Conv1D(128, 2, activation='relu')(x)
# layer-5: the second pooling layer
x = MaxPooling1D(pool_length = 2)(x)
# flatten layer
x = Flatten()(x)
# layer-6: the first dense layer
x = Dense(output_dim = 128, activation='relu')(x)
# layer-7: the second dense layer
x = Dense(output_dim = 128, activation='relu')(x)
# layer-8: the output layer
final_output = Dense(3, activation='sigmoid')(x)

# define the model
model2 = Model(input=sequence_input, output=final_output)

#view model
print(model2.summary())

# compile the model
model2.compile(loss='categorical_crossentropy', optimizer='adagrad', metrics=['acc'])
# training and validation
print('Training the model ...')
model2.fit(x=x_train, y=y_train, nb_epoch=15, batch_size=128, verbose=1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", filters=128, kernel_size=2)`
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=2)`
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=2)`
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=128)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=128)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: UserWarning: Update your `Model` call to the Keras 2 API: `

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 20)                0         
_________________________________________________________________
embedding_6 (Embedding)      (None, 20, 300)           804600    
_________________________________________________________________
conv1d_11 (Conv1D)           (None, 19, 128)           76928     
_________________________________________________________________
max_pooling1d_11 (MaxPooling (None, 9, 128)            0         
_________________________________________________________________
conv1d_12 (Conv1D)           (None, 8, 128)            32896     
_________________________________________________________________
max_pooling1d_12 (MaxPooling (None, 4, 128)            0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 512)               0   

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/15
395/395 [==============================] - 0s 707us/step - loss: 0.9573 - acc: 0.5570
Epoch 2/15
395/395 [==============================] - 0s 424us/step - loss: 0.8368 - acc: 0.5367
Epoch 3/15
395/395 [==============================] - 0s 435us/step - loss: 0.8077 - acc: 0.5013
Epoch 4/15
395/395 [==============================] - 0s 406us/step - loss: 0.7851 - acc: 0.5772
Epoch 5/15
395/395 [==============================] - 0s 418us/step - loss: 0.7205 - acc: 0.6127
Epoch 6/15
395/395 [==============================] - 0s 440us/step - loss: 0.5467 - acc: 0.7519
Epoch 7/15
395/395 [==============================] - 0s 426us/step - loss: 0.5882 - acc: 0.7620
Epoch 8/15
395/395 [==============================] - 0s 475us/step - loss: 0.2010 - acc: 0.9418
Epoch 9/15
395/395 [==============================] - 0s 417us/step - loss: 0.0559 - acc: 0.9873
Epoch 10/15
395/395 [==============================] - 0s 439us/step - loss: 0.0188 - acc: 0.9949
Epoch 11/15
395/395 [========

In [0]:
data_test_answers=pd.read_csv('drive/My Drive/dataset_test_answers.csv')
data_test_target1_answers=data_test_answers[data_test_answers["Target"]=="Climate Change is a Real Concern"]
y_ground_truth=data_test_target1_answers["Stance"].values;


y_test=np.zeros(len(data_test_target2))

for i in range(len(y_ground_truth)):
  if(y_ground_truth[i]=='AGAINST'):
    y_test[i]=0;
  elif(y_ground_truth[i]=='FAVOR'):
    y_test[i]=1;
  else:
    y_test[i]=2;


y_pred=model2.predict(x_test)

print(y_pred)

y=np.zeros(len(y_pred))
for i in range(len(y_pred)):
  y[i]=np.argmax(y_pred[i])

print(y_test)
print(y)

from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
print(f1_score(y,y_test,average='macro'))
print(precision_score(y,y_test,average='macro'))
print(recall_score(y,y_test,average='macro'))
print(accuracy_score(y,y_test))

[[1.58610046e-02 9.91029739e-02 3.75779152e-01]
 [4.64370847e-03 1.70566440e-02 6.75544143e-01]
 [1.63763762e-04 5.71032941e-01 1.08551979e-03]
 [4.59671021e-04 3.09045911e-02 2.02993870e-01]
 [2.45493484e-05 8.83932233e-01 2.57506435e-05]
 [2.03228865e-06 4.83223557e-01 1.05995799e-08]
 [2.55960226e-03 2.35084504e-01 2.60790586e-02]
 [1.28477812e-04 1.03536010e-01 8.65608454e-03]
 [4.54217196e-04 7.38176703e-03 6.37118459e-01]
 [6.28687758e-06 9.16883945e-01 5.94154335e-06]
 [7.34723153e-06 3.38228464e-01 4.78833914e-04]
 [7.63707764e-10 1.11084762e-06 9.71298337e-01]
 [1.71124935e-04 2.15690762e-01 7.44652748e-03]
 [3.95681345e-05 9.16492224e-01 2.00496888e-05]
 [2.19872594e-03 1.29851103e-02 6.33762598e-01]
 [3.08215618e-04 6.12463593e-01 6.10649586e-04]
 [2.87738442e-03 5.10182083e-02 1.26247314e-05]
 [5.12554398e-05 6.72774434e-01 3.41355801e-04]
 [1.49220228e-04 7.83922851e-01 5.67695097e-05]
 [8.02433169e-06 5.77841103e-02 2.64931619e-02]
 [1.17772818e-03 9.33566093e-02 1.250708

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [0]:
# for Target 3


target3_train_data=[]
for i in range(target1_train_len+target2_train_len,target1_train_len+target2_train_len+target3_train_len):
    target3_train_data.append(data_train_target3["Tweet"][i])
    
    
print(len(target3_train_data))

target3_train_data=convert_lowercase(target3_train_data)
target3_train_data=remove_digits(target3_train_data)
target3_train_data=remove_punctuations(target3_train_data)
target3_train_data=tokenize_data(target3_train_data)
target3_train_data=remove_stopwords(target3_train_data)
target3_train_data=lemmatize(target3_train_data)
target3_train_data=make_string(target3_train_data)


target3_test_data=[]
for i in range(target1_test_len+target2_test_len,target1_test_len+target2_test_len+target3_test_len):
    target3_test_data.append(data_test_target3["Tweet"][i])
    
target3_test_data=convert_lowercase(target3_test_data)
target3_test_data=remove_digits(target3_test_data)
target3_test_data=remove_punctuations(target3_test_data)
target3_test_data=tokenize_data(target3_test_data)
target3_test_data=remove_stopwords(target3_test_data)
target3_test_data=lemmatize(target3_test_data)
target3_test_data=make_string(target3_test_data)


y1=[]
for i in range(target1_train_len+target2_train_len,target1_train_len+target2_train_len+target3_train_len):
    y1.append(data_train_target3["Stance"][i])
  
print("Y1: ",len(y1))

target3_full_data=[]
target3_full_data.extend(target3_train_data)
target3_full_data.extend(target3_test_data)


# tokenize the words in the texts
max_nb_words=2000
tokenizer = Tokenizer(num_words = max_nb_words) 
tokenizer.fit_on_texts(target3_full_data) 
# convert each review text into a sequence of word-indices
matrix_word_indices = tokenizer.texts_to_sequences(target3_full_data)
# the dictionary for mapping a word to an index
dictionary_word_index = tokenizer.word_index

print(len(dictionary_word_index))





# pad each review text to a fixed length of word sequence
num_words_per_review=20
matrix_word_indices_fixed_length = pad_sequences(matrix_word_indices, maxlen = num_words_per_review)
# convert to numpy arrays 
data = np.array(matrix_word_indices_fixed_length)

print(matrix_word_indices_fixed_length.shape)

# allocation of training data and validation data
x_train = data[:len(data_train_target3)]
x_test = data[len(data_train_target3):]

y_train=np.zeros(shape=(target3_train_len,3))

print("y_train: ",len(y_train))

for i in range(len(y_train)):
  if(y1[i]=='AGAINST'):
    y_train[i][0]=1;
  elif(y1[i]=='FAVOR'):
    y_train[i][1]=1;
  else:
    y_train[i][2]=1;


print(len(x_train))
print(len(x_test))


# prepare embedding matrix
num_words = len(dictionary_word_index)
# embedding_matrix[0] is a all-zero vector representing no word
embedding_matrix = np.zeros((num_words+1, 300)) 

init_vector=np.zeros(300)


unknown_words=[]
for word, index in dictionary_word_index.items():
    if index > max_nb_words:
        continue 
    # get the glove vector for the word
    try:
      word_vector = word2vec.wv[word]
      if word_vector is not None: 
          embedding_matrix[index] = word_vector
    except:
      unknown_words.append(word)
      #print(word)
      embedding_matrix[index] = init_vector
  
print("The no. of unknown words: ", len(unknown_words))

print(len(embedding_matrix))





664
Y1:  664
3564
(949, 20)
y_train:  664
664
285
The no. of unknown words:  392
3565


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:101: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


In [0]:

# define the model
# layer 0: the input layer
num_words_per_tweet=20
sequence_input = Input(shape=(num_words_per_tweet,), dtype='int32')
# layer-1: the embedding layer
embedding_layer = Embedding(num_words+1, 300, weights=[embedding_matrix], input_length=num_words_per_tweet, trainable=True)
embedded_output = embedding_layer(sequence_input)
# layer-2: the first convolution layer
x = Conv1D(nb_filter=128, filter_length=2, activation='relu')(embedded_output)
# layer-3: the first pooling layer
x = MaxPooling1D(pool_length=2)(x)
# layer-4: the second convolution layer
x = Conv1D(128, 2, activation='relu')(x)
# layer-5: the second pooling layer
x = MaxPooling1D(pool_length = 2)(x)
# flatten layer
x = Flatten()(x)
# layer-6: the first dense layer
x = Dense(output_dim = 128, activation='relu')(x)
# layer-7: the second dense layer
x = Dense(output_dim = 128, activation='relu')(x)
# layer-8: the output layer
final_output = Dense(3, activation='sigmoid')(x)

# define the model
model2 = Model(input=sequence_input, output=final_output)

#view model
print(model2.summary())

# compile the model
model2.compile(loss='categorical_crossentropy', optimizer='adagrad', metrics=['acc'])
# training and validation
print('Training the model ...')
model2.fit(x=x_train, y=y_train, nb_epoch=10, batch_size=128, verbose=1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", filters=128, kernel_size=2)`
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=2)`
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=2)`
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=128)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=128)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: UserWarning: Update your `Model` call to the Keras 2 API: `

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 20)                0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 20, 300)           1069500   
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 19, 128)           76928     
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 9, 128)            0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 8, 128)            32896     
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 4, 128)            0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 512)               0   

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
664/664 [==============================] - 0s 564us/step - loss: 1.1683 - acc: 0.4940
Epoch 2/10
664/664 [==============================] - 0s 407us/step - loss: 0.9272 - acc: 0.4955
Epoch 3/10
664/664 [==============================] - 0s 398us/step - loss: 0.8240 - acc: 0.5331
Epoch 4/10
664/664 [==============================] - 0s 389us/step - loss: 0.6683 - acc: 0.7696
Epoch 5/10
664/664 [==============================] - 0s 387us/step - loss: 0.4405 - acc: 0.8178
Epoch 6/10
664/664 [==============================] - 0s 408us/step - loss: 0.3642 - acc: 0.8660
Epoch 7/10
664/664 [==============================] - 0s 401us/step - loss: 0.1086 - acc: 0.9669
Epoch 8/10
664/664 [==============================] - 0s 388us/step - loss: 0.0334 - acc: 0.9880
Epoch 9/10
664/664 [==============================] - 0s 399us/step - loss: 0.0137 - acc: 0.9985
Epoch 10/10
664/664 [==============================] - 0s 415us/step - loss: 0.0065 - acc: 1.0000


In [0]:
data_test_answers=pd.read_csv('drive/My Drive/dataset_test_answers.csv')
data_test_target3_answers=data_test_answers[data_test_answers["Target"]=="Feminist Movement"]
y_ground_truth=data_test_target3_answers["Stance"].values;


y_test=np.zeros(len(data_test_target3))

for i in range(len(y_ground_truth)):
  if(y_ground_truth[i]=='AGAINST'):
    y_test[i]=0;
  elif(y_ground_truth[i]=='FAVOR'):
    y_test[i]=1;
  else:
    y_test[i]=2;


y_pred=model2.predict(x_test)

print(y_pred)

y=np.zeros(len(y_pred))
for i in range(len(y_pred)):
  y[i]=np.argmax(y_pred[i])

print(y_test)
print(y)

from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
print(f1_score(y,y_test,average='macro'))
print(precision_score(y,y_test,average='macro'))
print(recall_score(y,y_test,average='macro'))
print(accuracy_score(y,y_test))

[[5.40152669e-01 7.31378794e-04 4.44587022e-06]
 [9.31704044e-01 4.60159617e-06 1.20612484e-07]
 [1.21366978e-03 6.56044483e-02 1.61850452e-03]
 [3.83973122e-04 2.89767981e-04 1.52915716e-04]
 [9.74312425e-01 6.53620009e-05 3.44804903e-07]
 [9.72890973e-01 1.34506822e-03 4.30535536e-08]
 [9.60862637e-03 2.67920659e-05 1.95205212e-04]
 [7.07274675e-03 1.78423584e-01 3.14652920e-04]
 [2.38484144e-03 5.05742431e-03 1.76292658e-03]
 [2.25779414e-03 8.69610429e-01 3.73928633e-05]
 [2.07187802e-01 3.41367722e-03 3.79413366e-03]
 [3.94731760e-04 4.99039888e-04 1.49857642e-05]
 [2.52896547e-02 4.97771180e-05 6.02618602e-05]
 [1.89149380e-03 9.34213400e-04 2.24536652e-05]
 [1.67713583e-01 6.91139698e-03 2.95996666e-04]
 [7.78257847e-04 1.48447752e-02 2.95096636e-03]
 [8.77995014e-01 2.62463618e-05 1.25437975e-04]
 [2.95006335e-02 2.11578012e-02 4.81158495e-04]
 [8.82627249e-01 5.28221790e-05 2.36699492e-07]
 [1.09701527e-04 2.61966765e-01 1.40277807e-05]
 [2.03448534e-03 2.55218148e-03 8.860856

In [0]:
# for Target 4


target4_train_data=[]
for i in range(target1_train_len+target2_train_len+target3_train_len,target1_train_len+target2_train_len+target3_train_len+target4_train_len):
    target4_train_data.append(data_train_target4["Tweet"][i])
    
    
print(len(target4_train_data))

target4_train_data=convert_lowercase(target4_train_data)
target4_train_data=remove_digits(target4_train_data)
target4_train_data=remove_punctuations(target4_train_data)
target4_train_data=tokenize_data(target4_train_data)
target4_train_data=remove_stopwords(target4_train_data)
target4_train_data=lemmatize(target4_train_data)
target4_train_data=make_string(target4_train_data)


target4_test_data=[]
for i in range(target1_test_len+target2_test_len+target3_test_len,target1_test_len+target2_test_len+target3_test_len+target4_test_len):
    target4_test_data.append(data_test_target4["Tweet"][i])
    
target4_test_data=convert_lowercase(target4_test_data)
target4_test_data=remove_digits(target4_test_data)
target4_test_data=remove_punctuations(target4_test_data)
target4_test_data=tokenize_data(target4_test_data)
target4_test_data=remove_stopwords(target4_test_data)
target4_test_data=lemmatize(target4_test_data)
target4_test_data=make_string(target4_test_data)


y1=[]
for i in range(target1_train_len+target2_train_len+target3_train_len,target1_train_len+target2_train_len+target3_train_len+target4_train_len):
    y1.append(data_train_target4["Stance"][i])
  
print("Y1: ",len(y1))

target4_full_data=[]
target4_full_data.extend(target4_train_data)
target4_full_data.extend(target4_test_data)


# tokenize the words in the texts
max_nb_words=2000
tokenizer = Tokenizer(num_words = max_nb_words) 
tokenizer.fit_on_texts(target4_full_data) 
# convert each review text into a sequence of word-indices
matrix_word_indices = tokenizer.texts_to_sequences(target4_full_data)
# the dictionary for mapping a word to an index
dictionary_word_index = tokenizer.word_index

print(len(dictionary_word_index))





# pad each review text to a fixed length of word sequence
num_words_per_review=20
matrix_word_indices_fixed_length = pad_sequences(matrix_word_indices, maxlen = num_words_per_review)
# convert to numpy arrays 
data = np.array(matrix_word_indices_fixed_length)

print(matrix_word_indices_fixed_length.shape)

# allocation of training data and validation data
x_train = data[:len(data_train_target4)]
x_test = data[len(data_train_target4):]

y_train=np.zeros(shape=(target4_train_len,3))

print("y_train: ",len(y_train))

for i in range(len(y_train)):
  if(y1[i]=='AGAINST'):
    y_train[i][0]=1;
  elif(y1[i]=='FAVOR'):
    y_train[i][1]=1;
  else:
    y_train[i][2]=1;


print(len(x_train))
print(len(x_test))


# prepare embedding matrix
num_words = len(dictionary_word_index)
# embedding_matrix[0] is a all-zero vector representing no word
embedding_matrix = np.zeros((num_words+1, 300)) 

init_vector=np.zeros(300)


unknown_words=[]
for word, index in dictionary_word_index.items():
    if index > max_nb_words:
        continue 
    # get the glove vector for the word
    try:
      word_vector = word2vec.wv[word]
      if word_vector is not None: 
          embedding_matrix[index] = word_vector
    except:
      unknown_words.append(word)
      #print(word)
      embedding_matrix[index] = init_vector
  
print("The no. of unknown words: ", len(unknown_words))

print(len(embedding_matrix))





639
Y1:  639
3353
(934, 20)
y_train:  639
639
295
The no. of unknown words:  470
3354


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:101: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


In [0]:

# define the model
# layer 0: the input layer
num_words_per_tweet=20
sequence_input = Input(shape=(num_words_per_tweet,), dtype='int32')
# layer-1: the embedding layer
embedding_layer = Embedding(num_words+1, 300, weights=[embedding_matrix], input_length=num_words_per_tweet, trainable=True)
embedded_output = embedding_layer(sequence_input)
# layer-2: the first convolution layer
x = Conv1D(nb_filter=128, filter_length=2, activation='relu')(embedded_output)
# layer-3: the first pooling layer
x = MaxPooling1D(pool_length=2)(x)
# layer-4: the second convolution layer
x = Conv1D(128, 2, activation='relu')(x)
# layer-5: the second pooling layer
x = MaxPooling1D(pool_length = 2)(x)
# flatten layer
x = Flatten()(x)
# layer-6: the first dense layer
x = Dense(output_dim = 128, activation='relu')(x)
# layer-7: the second dense layer
x = Dense(output_dim = 128, activation='relu')(x)
# layer-8: the output layer
final_output = Dense(3, activation='sigmoid')(x)

# define the model
model2 = Model(input=sequence_input, output=final_output)

#view model
print(model2.summary())

# compile the model
model2.compile(loss='categorical_crossentropy', optimizer='adagrad', metrics=['acc'])
# training and validation
print('Training the model ...')
model2.fit(x=x_train, y=y_train, nb_epoch=15, batch_size=128, verbose=1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", filters=128, kernel_size=2)`
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=2)`
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=2)`
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=128)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=128)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: UserWarning: Update your `Model` call to the Keras 2 API: `

Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 20)                0         
_________________________________________________________________
embedding_6 (Embedding)      (None, 20, 300)           1006200   
_________________________________________________________________
conv1d_11 (Conv1D)           (None, 19, 128)           76928     
_________________________________________________________________
max_pooling1d_11 (MaxPooling (None, 9, 128)            0         
_________________________________________________________________
conv1d_12 (Conv1D)           (None, 8, 128)            32896     
_________________________________________________________________
max_pooling1d_12 (MaxPooling (None, 4, 128)            0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 512)               0   

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/15
639/639 [==============================] - 0s 597us/step - loss: 1.0403 - acc: 0.4898
Epoch 2/15
639/639 [==============================] - 0s 382us/step - loss: 0.9292 - acc: 0.5649
Epoch 3/15
639/639 [==============================] - 0s 407us/step - loss: 0.8379 - acc: 0.5477
Epoch 4/15
639/639 [==============================] - 0s 391us/step - loss: 0.6219 - acc: 0.6886
Epoch 5/15
639/639 [==============================] - 0s 386us/step - loss: 0.5667 - acc: 0.7919
Epoch 6/15
639/639 [==============================] - 0s 384us/step - loss: 0.2079 - acc: 0.9703
Epoch 7/15
639/639 [==============================] - 0s 394us/step - loss: 0.2395 - acc: 0.9171
Epoch 8/15
639/639 [==============================] - 0s 401us/step - loss: 0.0444 - acc: 0.9937
Epoch 9/15
639/639 [==============================] - 0s 396us/step - loss: 0.0148 - acc: 0.9984
Epoch 10/15
639/639 [==============================] - 0s 387us/step - loss: 0.0088 - acc: 0.9984
Epoch 11/15
639/639 [========

In [0]:
data_test_answers=pd.read_csv('drive/My Drive/dataset_test_answers.csv')
data_test_target4_answers=data_test_answers[data_test_answers["Target"]=="Hillary Clinton"]
y_ground_truth=data_test_target4_answers["Stance"].values;


y_test=np.zeros(len(data_test_target4))

for i in range(len(y_ground_truth)):
  if(y_ground_truth[i]=='AGAINST'):
    y_test[i]=0;
  elif(y_ground_truth[i]=='FAVOR'):
    y_test[i]=1;
  else:
    y_test[i]=2;


y_pred=model2.predict(x_test)

print(y_pred)

y=np.zeros(len(y_pred))
for i in range(len(y_pred)):
  y[i]=np.argmax(y_pred[i])

print(y_test)
print(y)

from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
print(f1_score(y,y_test,average='macro'))
print(precision_score(y,y_test,average='macro'))
print(recall_score(y,y_test,average='macro'))
print(accuracy_score(y,y_test))

[[9.99502778e-01 1.82311339e-08 5.62471740e-08]
 [1.94254458e-01 5.73747456e-02 2.07147062e-01]
 [1.96396261e-01 4.82699275e-03 1.33617687e-05]
 [1.59476995e-02 1.36827528e-02 6.67095184e-04]
 [7.53741145e-01 4.19823045e-05 5.87811301e-05]
 [7.76960254e-01 1.16473362e-04 1.81505004e-06]
 [5.38189411e-02 8.87878414e-05 1.77936256e-02]
 [2.60030031e-02 5.19543886e-04 1.11278445e-01]
 [8.42754722e-01 9.12444739e-06 1.02768026e-05]
 [9.62349057e-01 5.93464320e-05 8.40666053e-06]
 [9.77709055e-01 6.75407227e-06 2.59505050e-05]
 [2.75975764e-02 1.94840133e-02 1.03854179e-01]
 [8.47435474e-01 1.14238262e-03 4.87139141e-06]
 [9.02150571e-02 2.56031752e-03 1.03523880e-01]
 [4.96215492e-01 1.19725764e-02 2.84077068e-05]
 [7.82671452e-01 4.34726477e-04 2.43127346e-04]
 [9.99438643e-01 6.55187335e-08 1.78836963e-08]
 [8.55516851e-01 1.08624808e-05 1.44944661e-05]
 [9.99605656e-01 3.88435240e-08 3.88094570e-08]
 [6.91772997e-02 2.34286994e-01 7.79062520e-06]
 [2.95582414e-03 1.05902851e-04 6.513785

In [0]:
# for Target 5


target5_train_data=[]
for i in range(target1_train_len+target2_train_len+target3_train_len+target4_train_len,target1_train_len+target2_train_len+target3_train_len+target4_train_len+target5_train_len):
    target5_train_data.append(data_train_target5["Tweet"][i])
    
    
print(len(target5_train_data))

target5_train_data=convert_lowercase(target5_train_data)
target5_train_data=remove_digits(target5_train_data)
target5_train_data=remove_punctuations(target5_train_data)
target5_train_data=tokenize_data(target5_train_data)
target5_train_data=remove_stopwords(target5_train_data)
target5_train_data=lemmatize(target5_train_data)
target5_train_data=make_string(target5_train_data)


target5_test_data=[]
for i in range(target1_test_len+target2_test_len+target3_test_len+target4_test_len,target1_test_len+target2_test_len+target3_test_len+target4_test_len+target5_test_len):
    target5_test_data.append(data_test_target5["Tweet"][i])
    
target5_test_data=convert_lowercase(target5_test_data)
target5_test_data=remove_digits(target5_test_data)
target5_test_data=remove_punctuations(target5_test_data)
target5_test_data=tokenize_data(target5_test_data)
target5_test_data=remove_stopwords(target5_test_data)
target5_test_data=lemmatize(target5_test_data)
target5_test_data=make_string(target5_test_data)


y1=[]
for i in range(target1_train_len+target2_train_len+target3_train_len+target4_train_len,target1_train_len+target2_train_len+target3_train_len+target4_train_len+target5_train_len):
    y1.append(data_train_target5["Stance"][i])
  
print("Y1: ",len(y1))

target5_full_data=[]
target5_full_data.extend(target5_train_data)
target5_full_data.extend(target5_test_data)


# tokenize the words in the texts
max_nb_words=2000
tokenizer = Tokenizer(num_words = max_nb_words) 
tokenizer.fit_on_texts(target5_full_data) 
# convert each review text into a sequence of word-indices
matrix_word_indices = tokenizer.texts_to_sequences(target5_full_data)
# the dictionary for mapping a word to an index
dictionary_word_index = tokenizer.word_index

print(len(dictionary_word_index))





# pad each review text to a fixed length of word sequence
num_words_per_review=20
matrix_word_indices_fixed_length = pad_sequences(matrix_word_indices, maxlen = num_words_per_review)
# convert to numpy arrays 
data = np.array(matrix_word_indices_fixed_length)

print(matrix_word_indices_fixed_length.shape)

# allocation of training data and validation data
x_train = data[:len(data_train_target5)]
x_test = data[len(data_train_target5):]

y_train=np.zeros(shape=(target5_train_len,3))

print("y_train: ",len(y_train))

for i in range(len(y_train)):
  if(y1[i]=='AGAINST'):
    y_train[i][0]=1;
  elif(y1[i]=='FAVOR'):
    y_train[i][1]=1;
  else:
    y_train[i][2]=1;


print(len(x_train))
print(len(x_test))


# prepare embedding matrix
num_words = len(dictionary_word_index)
# embedding_matrix[0] is a all-zero vector representing no word
embedding_matrix = np.zeros((num_words+1, 300)) 

init_vector=np.zeros(300)


unknown_words=[]
for word, index in dictionary_word_index.items():
    if index > max_nb_words:
        continue 
    # get the glove vector for the word
    try:
      word_vector = word2vec.wv[word]
      if word_vector is not None: 
          embedding_matrix[index] = word_vector
    except:
      unknown_words.append(word)
      #print(word)
      embedding_matrix[index] = init_vector
  
print("The no. of unknown words: ", len(unknown_words))

print(len(embedding_matrix))





603
Y1:  603
3133
(883, 20)
y_train:  603
603
280
The no. of unknown words:  468
3134


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:101: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).


In [0]:

# define the model
# layer 0: the input layer
num_words_per_tweet=20
sequence_input = Input(shape=(num_words_per_tweet,), dtype='int32')
# layer-1: the embedding layer
embedding_layer = Embedding(num_words+1, 300, weights=[embedding_matrix], input_length=num_words_per_tweet, trainable=True)
embedded_output = embedding_layer(sequence_input)
# layer-2: the first convolution layer
x = Conv1D(nb_filter=128, filter_length=2, activation='relu')(embedded_output)
# layer-3: the first pooling layer
x = MaxPooling1D(pool_length=2)(x)
# layer-4: the second convolution layer
x = Conv1D(128, 2, activation='relu')(x)
# layer-5: the second pooling layer
x = MaxPooling1D(pool_length = 2)(x)
# flatten layer
x = Flatten()(x)
# layer-6: the first dense layer
x = Dense(output_dim = 128, activation='relu')(x)
# layer-7: the second dense layer
x = Dense(output_dim = 128, activation='relu')(x)
# layer-8: the output layer
final_output = Dense(3, activation='sigmoid')(x)

# define the model
model2 = Model(input=sequence_input, output=final_output)

#view model
print(model2.summary())

# compile the model
model2.compile(loss='categorical_crossentropy', optimizer='adagrad', metrics=['acc'])
# training and validation
print('Training the model ...')
model2.fit(x=x_train, y=y_train, nb_epoch=15, batch_size=128, verbose=1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(activation="relu", filters=128, kernel_size=2)`
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=2)`
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: Update your `MaxPooling1D` call to the Keras 2 API: `MaxPooling1D(pool_size=2)`
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:17: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=128)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="relu", units=128)`
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: UserWarning: Update your `Model` call to the Keras 2 API: `

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         (None, 20)                0         
_________________________________________________________________
embedding_7 (Embedding)      (None, 20, 300)           940200    
_________________________________________________________________
conv1d_13 (Conv1D)           (None, 19, 128)           76928     
_________________________________________________________________
max_pooling1d_13 (MaxPooling (None, 9, 128)            0         
_________________________________________________________________
conv1d_14 (Conv1D)           (None, 8, 128)            32896     
_________________________________________________________________
max_pooling1d_14 (MaxPooling (None, 4, 128)            0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 512)               0   

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/15
603/603 [==============================] - 0s 659us/step - loss: 1.0056 - acc: 0.4876
Epoch 2/15
603/603 [==============================] - 0s 395us/step - loss: 0.9120 - acc: 0.6153
Epoch 3/15
603/603 [==============================] - 0s 417us/step - loss: 0.6031 - acc: 0.7612
Epoch 4/15
603/603 [==============================] - 0s 409us/step - loss: 0.5006 - acc: 0.7662
Epoch 5/15
603/603 [==============================] - 0s 445us/step - loss: 0.3726 - acc: 0.8474
Epoch 6/15
603/603 [==============================] - 0s 398us/step - loss: 0.1797 - acc: 0.9585
Epoch 7/15
603/603 [==============================] - 0s 457us/step - loss: 0.0659 - acc: 0.9867
Epoch 8/15
603/603 [==============================] - 0s 395us/step - loss: 0.3171 - acc: 0.9138
Epoch 9/15
603/603 [==============================] - 0s 409us/step - loss: 0.0440 - acc: 0.9950
Epoch 10/15
603/603 [==============================] - 0s 394us/step - loss: 0.0206 - acc: 0.9983
Epoch 11/15
603/603 [========

In [0]:
data_test_answers=pd.read_csv('drive/My Drive/dataset_test_answers.csv')
data_test_target5_answers=data_test_answers[data_test_answers["Target"]=="Legalization of Abortion"]
y_ground_truth=data_test_target5_answers["Stance"].values;


y_test=np.zeros(len(data_test_target5))

for i in range(len(y_ground_truth)):
  if(y_ground_truth[i]=='AGAINST'):
    y_test[i]=0;
  elif(y_ground_truth[i]=='FAVOR'):
    y_test[i]=1;
  else:
    y_test[i]=2;


y_pred=model2.predict(x_test)

print(y_pred)

y=np.zeros(len(y_pred))
for i in range(len(y_pred)):
  y[i]=np.argmax(y_pred[i])

print(y_test)
print(y)

from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, confusion_matrix
print(f1_score(y,y_test,average='macro'))
print(precision_score(y,y_test,average='macro'))
print(recall_score(y,y_test,average='macro'))
print(accuracy_score(y,y_test))
print(confusion_matrix(y,y_test))

[[8.95889103e-01 9.92275091e-06 2.89627189e-09]
 [2.61434436e-01 3.12231779e-02 1.89977884e-03]
 [9.96533573e-01 3.83648863e-10 1.04108978e-15]
 [9.58957553e-01 5.22887494e-05 6.55611757e-06]
 [9.61298347e-01 2.27782394e-07 1.27614003e-10]
 [1.65843964e-02 9.65031981e-03 3.51621310e-07]
 [3.61961126e-03 1.42875314e-03 8.48153908e-11]
 [5.00216186e-01 9.45558204e-05 2.97039747e-03]
 [9.55698609e-01 2.17379893e-05 2.80488189e-06]
 [8.49891901e-01 5.51640987e-04 5.96225262e-04]
 [2.96093434e-01 4.14916873e-03 2.93014348e-02]
 [1.80780888e-04 6.41381025e-01 2.53260136e-04]
 [6.70540035e-02 5.71742654e-03 2.77314186e-02]
 [7.44980574e-03 4.27612662e-03 3.58873358e-08]
 [1.90931559e-03 3.76043022e-01 4.60415758e-05]
 [9.90129650e-01 1.26211444e-05 3.11858548e-06]
 [9.83592629e-01 2.17172146e-06 2.20324779e-07]
 [9.64424670e-01 5.21762968e-06 1.75594828e-08]
 [5.80915809e-03 1.94353521e-01 6.20573759e-04]
 [5.67663789e-01 4.01631551e-05 3.65610708e-06]
 [2.61793554e-01 1.12536550e-03 4.358291